# The Battle of Neighborhoods (Week2)

# Contents:

## Introduction - Business Problem and Interested Audience

Staying here, living here, dreaming it over here, makes one wonder how close is your next city to your home. Any city in India, there are many things that can be availed like the back of your hand. So, when comparing a city like New York or Toronto one would be very mindful on which city feels like home by not only on the availability of the home made products but also the cost of living matters the most. I would like to explore the option in New York Manhattan as it would be close to place of work and have many eateries too. So, in search of an apartment, the requirements would be a single bedroom with rent not more than 5000 dollars per month. And should have Indian restaurants and grocery shopping with amneties such as gym and community centres and location should be near subways. Challenges are meeting the price, location and amenties in and around the apartment in New York Manhattan(as mentioned above).People all around the world who want to move cities, will have similar preferences of their own ethnicity and culture. Here, we could target the Asian audience even though it's applicable similarly to all who want to join a new company or settle in the city for a long time or look out on a job of thier preference or even business.

## Data description and Source 

Using Foursquare and geopy data to pinpoint the venues for the above data requirement. This includes the subway station, apartment price, neighborhood etc. Converion of location information to latitude and longitude.
Using this, we can solve the questions pertaining to rental pricing options, distance from subway to apartment to place of work. Also, the neighborhood and other data related to amneties and community centres.

So, we would need lat, long, borough, neighborhood information as columns in the dataframe of New York. The subway related information s gathered from the wikipedia page for web scraping - https://en.wikipedia.org/wiki/List_of_New_York_City_Subway_stations_in_Manhattan
A csv file is obtained to represent the data as mentioned above and the data is mapped.


## Methodology

Here, we answer the questions - Details of the apartment including price, address, single bedroom & venues
Subway stations details & venues
Neighbourhood details including the amenties & venues\ Use Wikipedia to gather the details

In [1]:
import numpy as np 
import time
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
import requests 
from pandas.io.json import json_normalize 

from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [2]:
address = 'India Gate, Delhi'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Indian city {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Indian city 28.6128806, 77.2295975.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [3]:
CLIENT_ID = 'F5550W10YWYY3GC5JOT3S31A4FX5YOG2OWKRRUYJZCZG5HCL' 
CLIENT_SECRET = 'V3Y0M533TBT115CPQGYZEWWWM5W22STDFOUBCMWICS4EJ3SO' 
VERSION = '20180605'

In [4]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=F5550W10YWYY3GC5JOT3S31A4FX5YOG2OWKRRUYJZCZG5HCL&client_secret=V3Y0M533TBT115CPQGYZEWWWM5W22STDFOUBCMWICS4EJ3SO&v=20180605&ll=28.6128806,77.2295975&radius=500&limit=100'

In [5]:
results = requests.get(url).json()

In [6]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Amar Jawan Jyoti | अमर जवान ज्योति (Amar Jawan...,Sculpture Garden,28.612980,77.228247
1,India Gate | इंडिया गेट (India Gate),Monument / Landmark,28.612796,77.229207
2,UPSC Chaat,Snack Place,28.609324,77.227879
3,Veg Gulati Restaurant,Vegetarian / Vegan Restaurant,28.609089,77.230743
4,Children's Park,Playground,28.610664,77.230816
5,National War Memorial,Memorial Site,28.613303,77.232579
6,UPSC Lane,Smoke Shop,28.609346,77.227874
7,Chor Bizzare,Indian Restaurant,28.608837,77.229894
8,Lovely Chuski,Park,28.612823,77.224908


In [8]:
#Map of Indian residence
latitude=28.6128806 
longitude=77.2295975
# create map of Singapore place  using latitude and longitude values
map_in = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_in)  
    
map_in

Below shows us to identify the cluster venues around Manhattan.

In [ ]:
#Map of Manhattan
latitude= 40.7308619
longitude= -73.9871558 

kclusters=5
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
  # add markers for rental places to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)    
    
       
map_clusters

Below shows us the geodata of a place of rent in Manhattan

In [ ]:
mh_rent=pd.read_csv('MH_flats_price.csv')
for n in range(len(mh_rent)):   
    address= mh_rent['Address'][n]
    address=(mh_rent['Address'][n]+ '  , '+' Manhattan NY ')
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    mh_rent['Lat'][n]=latitude
    mh_rent['Long'][n]=longitude

# save dataframe to csv file
mh_rent.to_csv('MH_rent_latlong.csv',index=False)
mh_rent.shape

# create map of Manhattan using latitude and longitude values from Nominatim
latitude= 40.7308619
longitude= -73.9871558

map_manhattan_rent = folium.Map(location=[latitude, longitude], zoom_start=12.5)

# add markers to map
for lat, lng, label in zip(mh_rent['Lat'], mh_rent['Long'],'$ ' + mh_rent['Rent_Price'].astype(str)+ ',  '+ mh_rent['Address']):      
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_rent) 
map_manhattan_rent

The above should be done for subways and amenities nearby

## Result

Getting all the info to one map of subways, rentals etc

In [ ]:
latitude= 40.7308619
longitude= -73.9871558

map_mh_one = folium.Map(location=[latitude, longitude], zoom_start=13.3)

# add markers to map
for lat, lng, label in zip(mh_rent['Lat'], mh_rent['Long'],'$ ' + mh_rent['Rent_Price'].astype(str)+ ', '+mh_rent['Address']):      
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mh_one) 
    
    # add markers of subway locations to map
for lat, lng, label in zip(mhsub1['lat'], mhsub1['long'],  mhsub1['sub_station'].astype(str) ):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=6,
        radius=6,
        popup=label,
        color='red',
        fill_color='red',
        fill_opacity=2.5,
    ).add_to(map_mh_one) 


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_mh_one)

    #Then measurement can be added here

After the code is run, we can find the places affordable with the first apartment in Manhattan Park, 10-40 River Rd, New York and the second one in 
The Grand, 1770 Grand Concourse, Bronx, NY.
Using kmeans we can get the venue of the cluster.

## Discussion

From above, the Grand apartment has $1849 which is very much inside the budget along with the John's Boxing Gym.
Manhattan Park apartment has pricing upto $3200 with the Hospital near to it.

## Conclusion

Conisedring the above, we would like to prefer The Grand hotel because of the ease of travel from workplace, low cost and a gym nearby to hop on
anytime. Hence, as a data scientist we would be able to conclude the number of options to choose from cluster of venues and make a suitable
decision with great ease.